In [1]:
%pylab inline
import pandas as pd
import seaborn

%config InlineBackend.figure_format = 'svg'
seaborn.set_style("white")

Populating the interactive namespace from numpy and matplotlib


In [2]:
df1 = pd.read_csv('data/turnstile_180623.txt')
df2 = pd.read_csv('data/turnstile_180602.txt')
df1.head(3)

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,06/16/2018,00:00:00,REGULAR,6658725,2257032
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,06/16/2018,04:00:00,REGULAR,6658747,2257036
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,06/16/2018,08:00:00,REGULAR,6658766,2257068


In [3]:
# df_alldata = pd.concat([df1,df2])

Check hierarchy of members

In [4]:
# df1.groupby(['STATION', 'UNIT', 'C/A'])['SCP'].unique().head(5)

Strip whitespace in columns

In [5]:
newcolumns = df1.columns.str.strip()
df1.columns = newcolumns

In [6]:
df1['DATETIME'] = df1['DATE'] + ' ' + df1['TIME']
df1['DATETIME'] = pd.to_datetime(df1['DATETIME'])
df1.sample(3)

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,DATETIME
188556,R617,R058,00-00-00,BERGEN ST,23,IRT,06/16/2018,16:00:00,REGULAR,8408044,2067337,2018-06-16 16:00:00
76497,N309A,R140,00-06-03,QUEENS PLAZA,EMR,IND,06/17/2018,17:00:00,REGULAR,11877,103115,2018-06-17 17:00:00
25181,D008,R392,00-06-00,18 AV,N,BMT,06/18/2018,21:00:00,REGULAR,2098293,964157,2018-06-18 21:00:00


In [7]:
df1['ENTRY_DIFFS'] = (df1
                      .sort_values(by=['STATION','UNIT', 'C/A', 'SCP'])[['STATION','UNIT', 'C/A', 'SCP', 'ENTRIES', 'EXITS', 'DATETIME']]
                      .groupby(['STATION', 'UNIT', 'C/A', 'SCP'])['ENTRIES']
                      .diff())
df1['EXIT_DIFFS'] = (df1
                     .sort_values(by=['STATION','UNIT', 'C/A', 'SCP'])[['STATION','UNIT', 'C/A', 'SCP', 'ENTRIES', 'EXITS', 'DATETIME']]
                     .groupby(['STATION', 'UNIT', 'C/A', 'SCP'])['EXITS']
                     .diff())

In [8]:
# check to make sure NaN appears for the first row for an SCP
df1[40:].head(5)

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,DATETIME,ENTRY_DIFFS,EXIT_DIFFS
40,A002,R051,02-00-00,59 ST,NQR456W,BMT,06/22/2018,16:00:00,REGULAR,6666235,2259814,2018-06-22 16:00:00,329.0,74.0
41,A002,R051,02-00-00,59 ST,NQR456W,BMT,06/22/2018,20:00:00,REGULAR,6666924,2259874,2018-06-22 20:00:00,689.0,60.0
42,A002,R051,02-00-01,59 ST,NQR456W,BMT,06/16/2018,00:00:00,REGULAR,5964171,1334598,2018-06-16 00:00:00,NaN,NaN
43,A002,R051,02-00-01,59 ST,NQR456W,BMT,06/16/2018,04:00:00,REGULAR,5964188,1334606,2018-06-16 04:00:00,17.0,8.0
44,A002,R051,02-00-01,59 ST,NQR456W,BMT,06/16/2018,08:00:00,REGULAR,5964202,1334630,2018-06-16 08:00:00,14.0,24.0


In [9]:
df1.sort_values(by=['STATION','UNIT', 'C/A', 'SCP'])[['STATION','UNIT', 'C/A', 'SCP', 'ENTRIES', 'EXITS', 'DATETIME', 'ENTRY_DIFFS']].groupby(['STATION', 'UNIT', 'C/A', 'SCP'])['ENTRY_DIFFS'].sum()

STATION          UNIT  C/A   SCP     
1 AV             R248  H007  00-00-00    24023.0
                             00-00-01    31421.0
                             00-03-00     6240.0
                             00-03-01     4825.0
                             00-03-02     8038.0
                       H008  01-00-00     1775.0
                             01-00-01     4915.0
                             01-00-02    10617.0
                             01-00-03    19767.0
                             01-00-04    24013.0
103 ST           R180  R252  00-00-00    18237.0
                             00-00-01    15986.0
                             00-00-02    14905.0
                             00-03-00     9889.0
                             00-03-01     9485.0
                             00-03-02    10311.0
                 R191  R170  00-00-00    19710.0
                             00-00-01    13257.0
                             00-00-02    14605.0
                             00